## **Fake News Classifier using Bi-drectional LSTM**

Mount drive to read kaggle files from drive

In [5]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 9.3MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 450kB 46.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=cf590136764d0bfb4271d344d99202459d3c4c4fd068021ddd0f7fb82ab93c11
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing inst

In [45]:
import pandas as pd
import numpy as np

import tensorflow as tf
tf.__version__     # have keras imbedded into tensorflow
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences # to make sentenses of equal size
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional  # to make LSTM RNN bidirectional 
from tensorflow.keras.layers import Dropout

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [7]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FakeNews_NLP_BiLSTM/fake-news/train.csv')

In [8]:
df.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [9]:
df.shape

(20800, 5)

# Cleaing the dataset

In [10]:
# dropping Nan values
df = df.dropna()

In [11]:
df.shape

(18285, 5)

In [12]:
# sperating dependent feature (label column) and independent features 
x = df.drop('label', axis=1)
y = df['label']

In [13]:
# checking weather the dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [14]:
x.shape

(18285, 4)

In [16]:
### Vocabulary size to specify dictionary size
voc_size = 5000

In [17]:
sms = x.copy()

In [18]:
sms['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
sms.reset_index(inplace=True)

In [21]:
# using nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Data processing**


*   Removing specail charaters 
*   Lower case the text/message/news


*   Spliting by removing the stop words
*   Steming of the news





In [25]:
## data preprocessing 
ps = PorterStemmer()
corpus = []
for i in range(0 , len(sms)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', sms['title'][i]) # removing all special charaters except a-z and A-Z
  review = review.lower() #lower casing the text
  review = review.split() # split the sentense into individual words 

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # removing stop words
  review = ' '.join(review) # joining back the sentense after removing special charaters, stopswords etc
  corpus.append(review) # putting back the processed sentense into review 

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [26]:
# view all the words after processing 
corpus 

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

# Assigning index to all words
Using one_hot to assign index based on the vocabulary size  

In [27]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[1246, 2524, 2980, 848, 652, 4203, 4585, 2435, 4165, 4924],
 [3765, 3377, 3766, 864, 445, 421, 3728],
 [1560, 1283, 4134, 1114],
 [4320, 1439, 4493, 3836, 2632, 1269],
 [703, 445, 3131, 1994, 1507, 2287, 445, 3617, 224, 3396],
 [3447,
  3612,
  3465,
  3686,
  1102,
  1150,
  2419,
  2251,
  3521,
  2351,
  3262,
  3509,
  3308,
  4058,
  3728],
 [1078, 3260, 988, 3162, 4613, 536, 4305, 505, 3873, 1696, 1685],
 [2114, 745, 2937, 2877, 3358, 4469, 1150, 1142, 3873, 1696, 1685],
 [4075, 653, 4074, 2511, 1843, 2848, 1879, 4831, 1150, 2091],
 [3752, 3234, 326, 3475, 2445, 2073, 4378, 3021],
 [4018, 1433, 3832, 138, 194, 1749, 3404, 4083, 1129, 4777, 1980],
 [3836, 29, 652, 2848, 1150, 3358],
 [3702, 2741, 3535, 2470, 3273, 1655, 630, 1594, 3340],
 [3192, 4430, 4782, 312, 1249, 1948, 3531, 3873, 1696, 1685],
 [2706, 2814, 3648, 2164, 3680, 3873, 1696, 1685],
 [1731, 2804, 639, 2103, 3132, 4896, 3767, 4679, 2038, 3474],
 [4156, 3986, 3377],
 [4105, 2638, 1863, 2244, 1150, 3011, 2268, 3728],

# **Embedding Representation**
making all sentenses of same length 

In [28]:
sms_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sms_length)
print(embedded_docs)

[[   0    0    0 ... 2435 4165 4924]
 [   0    0    0 ...  445  421 3728]
 [   0    0    0 ... 1283 4134 1114]
 ...
 [   0    0    0 ... 3873 1696 1685]
 [   0    0    0 ...  539  685  189]
 [   0    0    0 ... 4165 2016 3777]]


In [29]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1246,
       2524, 2980,  848,  652, 4203, 4585, 2435, 4165, 4924], dtype=int32)

# **Model Creation** 
LSTM model

In [46]:
embedding_vector_feature = 40  #coverting embedded doc into a vector of 40, this will now take 40 features
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sms_length))
model.add(Bidirectional(LSTM(100)))  # 2 lstm is build 
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
len(embedded_docs), y.shape

(18285, (18285,))

In [33]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

# Spliting the data into test and train part

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(x_final, y_final, test_size = 0.35, random_state = 42)

# **Training the model**

In [47]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)

Train on 11885 samples, validate on 6400 samples
Epoch 1/20
11885/11885 [==============================] - 7s 575us/sample - loss: 0.3126 - accuracy: 0.8502 - val_loss: 0.1991 - val_accuracy: 0.9170
Epoch 2/20
11885/11885 [==============================] - 3s 253us/sample - loss: 0.1394 - accuracy: 0.9480 - val_loss: 0.2305 - val_accuracy: 0.9097
Epoch 3/20
11885/11885 [==============================] - 3s 256us/sample - loss: 0.0972 - accuracy: 0.9659 - val_loss: 0.2191 - val_accuracy: 0.9105
Epoch 4/20
11885/11885 [==============================] - 3s 255us/sample - loss: 0.0699 - accuracy: 0.9755 - val_loss: 0.2849 - val_accuracy: 0.9130
Epoch 5/20
11885/11885 [==============================] - 3s 257us/sample - loss: 0.0470 - accuracy: 0.9845 - val_loss: 0.3241 - val_accuracy: 0.9141
Epoch 6/20
11885/11885 [==============================] - 3s 257us/sample - loss: 0.0319 - accuracy: 0.9905 - val_loss: 0.4404 - val_accuracy: 0.9080
Epoch 7/20
11885/11885 [===========================

# **Model Performace**

In [48]:
y_pred = model.predict_classes(X_test)

In [49]:
confusion_matrix(Y_test, y_pred)

array([[3305,  328],
       [ 262, 2505]])

In [50]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3633
           1       0.88      0.91      0.89      2767

    accuracy                           0.91      6400
   macro avg       0.91      0.91      0.91      6400
weighted avg       0.91      0.91      0.91      6400



# **Result**
## **Accuracy: 91%**